In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf


# Set a fixed random seed value, for reproducibility
SEED = 2137
np.random.seed(SEED)
tf.random.set_seed(SEED)

# list of exercises files
EXERCISES = [
    "biceps_curl_1",
    "triceps_ext",
    "noise2",
    "shoulder_side",
    "shoulder_front"
]
#SPECIFY DATA LOCATION
dataset_path = "/home/dev/data"

SAMPLES_PER_EXERCISES = 297

NUM_OF_EXERCISES = len(EXERCISES)

# output matrix
ONE_HOT_ENCODED_EXERCISES = np.eye(NUM_OF_EXERCISES)

inputs = []
outputs = []

# reading every csv file and putting it into input and output
for exercise_index in range(NUM_OF_EXERCISES):
  gesture = EXERCISES[exercise_index]
  print(f"Processing index {exercise_index} for gesture '{gesture}'.")
  
  output = ONE_HOT_ENCODED_EXERCISES[exercise_index]
  
  df = pd.read_csv(dataset_path + gesture + ".csv")
  
  # calculate the number of gesture recordings in the file
  num_recordings = int(df.shape[0] / SAMPLES_PER_EXERCISES)
  
  print(f"\tThere are {num_recordings} recordings of the {gesture} exercise.")
  
  for i in range(num_recordings):
    tensor = []
    for j in range(SAMPLES_PER_EXERCISES):
      index = i * SAMPLES_PER_EXERCISES + j
      # normalize the input data, between 0 to 1:
      # - acceleration is between: -4 to +4
      # - gyroscope is between: -2000 to +2000
      tensor += [
          (df['aX'][index] + 4) / 8,
          (df['aY'][index] + 4) / 8,
          (df['aZ'][index] + 4) / 8,
          (df['gX'][index] + 2000) / 4000,
          (df['gY'][index] + 2000) / 4000,
          (df['gZ'][index] + 2000) / 4000
      ]

    inputs.append(tensor)
    outputs.append(output)

inputs = np.array(inputs)
outputs = np.array(outputs)


In [ ]:
# Randomize the order of the inputs, so they can be evenly distributed for training, testing, and validation
num_inputs = len(inputs)
randomize = np.arange(num_inputs)
np.random.shuffle(randomize)
# Swap the consecutive indexes with the randomized indexes
inputs = inputs[randomize]
outputs = outputs[randomize]
# Split into three sets: training, testing and validation
TRAIN_SPLIT = int(0.6 * num_inputs)
TEST_SPLIT = int(0.2 * num_inputs + TRAIN_SPLIT)
inputs_train, inputs_test, inputs_validate = np.split(inputs, [TRAIN_SPLIT, TEST_SPLIT])
outputs_train, outputs_test, outputs_validate = np.split(outputs, [TRAIN_SPLIT, TEST_SPLIT])

print("done")

In [ ]:
# build the model and train it
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(50, activation='relu')) 
model.add(tf.keras.layers.Dense(NUM_OF_EXERCISES, activation='softmax')) 
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
history = model.fit(inputs_train, outputs_train, epochs=700, batch_size=1, validation_data=(inputs_validate, outputs_validate))
model.save('/home/trep/data/keras_model')

In [ ]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)

#optimizing
#converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()

open("exercise_model.tflite", "wb").write(tflite_model)
  
import os
basic_model_size = os.path.getsize("exercise_model.tflite")
print("Model is %d bytes" % basic_model_size)

In [ ]:
!echo "const unsigned char model[] = {" > /home/dev/data/model.h
!cat exercise_model.tflite | xxd -i      >> /home/dev/data/model.h
!echo "};"                              >> /home/dev/data/model.h

import os
model_h_size = os.path.getsize("model.h")
print(f"Header file, model.h, is {model_h_size:,} bytes.")